In [1]:
%load_ext autoreload
%autoreload 2

# make station graph
다루기 쉽게(?) 그래프로 만들어줍니당

## Load Data

In [2]:
import pandas as pd
import util
from stationUtil import *

In [3]:
df = util.loadPickle(util.PATH_INTER_STATION_SPEND_TIME)
subway_graph_index, subway_graph = fn_export_subway_graph(df)
exchange_relation_index = fn_export_exchange_station_index(idx=subway_graph_index)

print(*exchange_relation_index)
subway_graph[exchange_relation_index[0]]

len(subway_graph) == len(interval_time) :  True
(119, 219, True) (123, 305, True) (125, 402, True) (208, 301, True) (219, 119, True) (233, 313, True) (301, 208, True) (305, 123, True) (309, 401, True) (313, 233, True) (401, 309, True) (402, 125, True)


{'stoppingTime': 0, 'time': 120, 'dist': 1}

In [4]:
interval = util.loadPickle(util.PATH_WEIGHTED_MEAN_INTERVAL)

## Add Weight `PATH_WEIGHTED_MEAN_INTERVAL`

In [5]:
new_subway_graph = {}
for index, value in subway_graph.items():
    value["dispatchingTime"] = interval[ index[0] ]
    new_subway_graph[index] = value

new_subway_graph[(96, 95, False)]

{'stoppingTime': 20,
 'time': 180,
 'dist': 14,
 'dispatchingTime': 354.64715877968746}

In [6]:
# GPT-4
# Prompt1:인접리스트로 구현한 가중치 그래프를 파이썬으로 구현 해줘
# Prompt2:구현한 그래프로 가중치의 합이 최소가되는 경로를 추출하는 함수를 짜줘
import heapq


class WeightedGraph:
    def __init__(self):
        self.adj_list = {}

    def add_vertex(self, vertex):
        if vertex not in self.adj_list:
            self.adj_list[vertex] = []

    def add_edge(self, from_vertex, to_vertex, weight):
        if from_vertex in self.adj_list:
            self.adj_list[from_vertex].append((to_vertex, weight))
        else:
            self.adj_list[from_vertex] = [(to_vertex, weight)]

    def print_graph(self):
        for vertex in self.adj_list:
            print(f"{vertex} -> {self.adj_list[vertex]}")

    def find_min_path(self, start_vertex, end_vertex):
        distances = {vertex: float("infinity") for vertex in self.adj_list}
        distances[start_vertex] = 0
        previous_vertices = {vertex: None for vertex in self.adj_list}
        pq = [(0, start_vertex)]

        while pq:
            current_distance, current_vertex = heapq.heappop(pq)

            if current_vertex == end_vertex:
                break

            for neighbor, weight in self.adj_list[current_vertex]:
                distance = current_distance + weight

                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_vertices[neighbor] = current_vertex
                    heapq.heappush(pq, (distance, neighbor))

        path, current_vertex = [], end_vertex
        while previous_vertices[current_vertex] is not None:
            path.append(current_vertex)
            current_vertex = previous_vertices[current_vertex]
        if path:
            path.append(start_vertex)

        return path[::-1]

    # Prompt 3: ```{Above Lines}``` 위 코드는 인접리스트로 구현된 그래프를 구현한 모습이다. 메서드 find_min_path에서 나온 출력 값(리스트로된 경로)을 이용해 해당경로의 가중치의 총합을 구하는 메서드를 추가로 작성해줘
    def calculate_path_weight(self, path):
        total_weight = 0
        if len(path) < 2:
            return total_weight

        for i in range(len(path) - 1):
            from_vertex = path[i]
            to_vertex = path[i + 1]
            for neighbor, weight in self.adj_list[from_vertex]:
                if neighbor == to_vertex:
                    total_weight += weight
                    break

        return total_weight

In [7]:
# Draw Graph

def time_weight_graph(exclude_list: list = [], new_subway_graph=new_subway_graph):
    graph = WeightedGraph()
    for index, weights in new_subway_graph.items():
        v1, v2, isTransfer = index
        graph.add_vertex(v1)
        graph.add_vertex(v2)
        time_spend = weights["time"]

        # 시나리오 작성 시, 정차하지 않는 역은 정차 시간(stopTime)을 빼고 간선을 추가한다.
        if v1 in exclude_list:
            graph.add_edge(v1, v2, time_spend)
            continue

        time_spend = weights["time"] + weights["stoppingTime"]

        # 환승하는 경우 (최악) 배차시간을 적용. - 최소 환승을 적용하기 위해서임.
        if isTransfer:
            time_spend += weights["dispatchingTime"]

        graph.add_edge(v1, v2, time_spend)

    return graph


def dist_weight_graph(new_subway_graph=new_subway_graph):
    graph = WeightedGraph()
    for index, weights in new_subway_graph.items():
        v1, v2, _ = index
        graph.add_vertex(v1)
        graph.add_vertex(v2)
        graph.add_edge(v1, v2, weights["dist"])

    return graph

In [8]:
#Applicated Function

def calc_total_path_weight(g: WeightedGraph, sCode_start, sCode_end):
    weight = 1  # 단위를 시간으로 바꾸고 싶으면 1/3600 으로 할것
    """
    calc_weight 최단경로를 지났을 때의 weigt 총합을 반환함.

    Args:
        g (WeightedGraph): 그래프 넣으면됨
        sCode_start (_type_): 시작지점
        sCode_end (_type_): 끝지점

    Returns:
        _type_: 초 단위로 출력함.
    """
    path = g.find_min_path(sCode_start, sCode_end)
    return g.calculate_path_weight(path) * weight


def calc_line_speed(line_num: int, g_dist: WeightedGraph, g_time: WeightedGraph):
    UNITFACTOR_P1KM_TO_1KM = 1 / 10
    UNITFACTOR_1SEC_TO_1HOUR = 1 / 3600
    path = []
    """
    calc_line_speed 표정속도 구하는 함수임.

    Args:
        line_num (int): 호선입력하세요
        g_dist (WeightedGraph): dist graph 입력하세요
        g_time (WeightedGraph): time graph 입력하세요

    Returns:
        float: 표정속도[km/h]
    """
    if line_num == 1:
        path = [*range(95, 134 + 1)]
    elif line_num == 2:
        path = [*range(201, 243 + 1)]
    elif line_num == 3:
        path = [*range(301, 317 + 1)]
    elif line_num == 4:
        path = [*range(401, 414 + 1)]
    return (g_dist.calculate_path_weight(path) * UNITFACTOR_P1KM_TO_1KM) / (
        g_time.calculate_path_weight(path) * UNITFACTOR_1SEC_TO_1HOUR
    )

In [9]:
# 아무것도 제외 하지 않으면 완행열차와 동일하다.
UNITFACTOR_P1KM_TO_1KM = 1 / 10
UNITFACTOR_1SEC_TO_1HOUR = 1 / 3600
EXCLUDE_STATION = []
g_time = time_weight_graph(exclude_list = EXCLUDE_STATION)
g_dist = dist_weight_graph()

sCode_start,sCode_end = 95,126
calc_total_path_weight(g_time,sCode_start,sCode_end) , calc_total_path_weight(g_dist,sCode_start,sCode_end) # (Second, 0.1km)

(3835, 316)

In [10]:
# 완행열차 Test
(   # 표정속도                           # 실제 표정속도
    calc_line_speed(1,g_dist,g_time),   # 30.7㎞/h
    calc_line_speed(2,g_dist,g_time),   # 32.3㎞/h
    calc_line_speed(3,g_dist,g_time),   # 31.9㎞/h
    calc_line_speed(4,g_dist,g_time)    # 28.8㎞/h
)

(30.271865121180195, 32.28571428571429, 31.63106796116505, 29.488054607508534)

In [11]:
# 건너뛸 지점을 지정 했으므로 급행열차이다.
EXCLUDE_STATION = [120,121,122,123]
g_time = time_weight_graph(exclude_list = EXCLUDE_STATION)
g_dist = dist_weight_graph()

sCode_start,sCode_end = 95,126
calc_total_path_weight(g_time,sCode_start,sCode_end) , calc_total_path_weight(g_dist,sCode_start,sCode_end) # (Second, 0.1km)

(3695, 316)

In [12]:
# 급행열차 Test
(   # 표정속도                           # 실제 표정속도
    calc_line_speed(1,g_dist,g_time),   # 30.7㎞/h  # 1호선의 표정속도가 증가 했음을 알 수 있다!!!
    calc_line_speed(2,g_dist,g_time),   # 32.3㎞/h
    calc_line_speed(3,g_dist,g_time),   # 31.9㎞/h
    calc_line_speed(4,g_dist,g_time)    # 28.8㎞/h
)

(31.19218241042346, 32.28571428571429, 31.63106796116505, 29.488054607508534)